In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification


X, y = make_classification(
    n_samples=1000,
    n_features=4,    
    n_informative=4, 
    n_redundant=0,   
    n_repeated=0,    
    n_classes=2,     
    random_state=42  
)

# Convert to a DataFrame for better visualization and manipulation
columns = [f"Feature_{i+1}" for i in range(4)]
df = pd.DataFrame(X, columns=columns)
df['Quality'] = y  # Add the target column

# Save the dataset as a CSV file
df.to_csv("binary_classification_dataset.csv", index=False)

from sklearn.preprocessing import MinMaxScaler

df.loc[df["Quality"] == "good", "Quality"] = 1
df.loc[df["Quality"] == "bad", "Quality"] = 0

scaler = MinMaxScaler(feature_range=(0,1))
scaled_x = scaler.fit_transform(df.iloc[:,:4])

x_train = scaled_x[:850]
x_test = scaled_x[850:]

y_train = df.iloc[:850,4].to_numpy(dtype= np.float32)
y_test = df.iloc[850:,4].to_numpy(dtype= np.float32)

In [2]:
import qiskit
import qiskit.circuit
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.optimizers import L_BFGS_B, ADAM
from qiskit_machine_learning.algorithms.classifiers import VQC
from matplotlib import pyplot as plt

num_features = 4

feature_map = ZZFeatureMap(num_features,1)
ansatz = TwoLocal(num_features, entanglement="linear",rotation_blocks = qiskit.circuit.library.RYGate, entanglement_blocks= "cx", reps=6)


objective_func_vals = []

opt = L_BFGS_B()
step = 0
def print_fn(_,loss):
    global step
    step += 1
    print(step, loss)

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=opt,
    callback= print_fn
)

vqc.fit(x_train,y_train)

KeyboardInterrupt: 

In [ ]:
train_score = vqc.score(x_train, y_train)
test_score = vqc.score(x_test,y_test)
print(train_score)
print(test_score)

In [ ]:
optimal_parameters = vqc.weights
vqc.interpret(4,2)

vqc._circuit.decompose().draw(output="mpl")